<a href="https://colab.research.google.com/github/for-x-tears/MLhomeworkLU/blob/main/emotions_prologue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers torch pandas tqdm openpyxl safetensors

print("✓ Зависимости установлены")

import pandas as pd
from pathlib import Path
from transformers import pipeline
import warnings
from tqdm import tqdm
import torch

warnings.filterwarnings('ignore')

if torch.cuda.is_available():
    print(f"✓ GPU доступен: {torch.cuda.get_device_name(0)}")
    print(f"✓ CUDA версия: {torch.version.cuda}")
else:
    print("⚠ GPU не обнаружен, будет использоваться CPU")

EMOTION_MAP = {
    0: 'joy',
    1: 'interest',
    2: 'surprise',
    3: 'sadness',
    4: 'anger',
    5: 'disgust',
    6: 'fear',
    7: 'guilt',
    8: 'neutral',
    9: 'average'
}

EMOTION_STR_MAP = {
    'LABEL_0': 'joy',
    'LABEL_1': 'interest',
    'LABEL_2': 'surprise',
    'LABEL_3': 'sadness',
    'LABEL_4': 'anger',
    'LABEL_5': 'disgust',
    'LABEL_6': 'fear',
    'LABEL_7': 'guilt',
    'LABEL_8': 'neutral',
    'LABEL_9': 'average',
    '0': 'joy',
    '1': 'interest',
    '2': 'surprise',
    '3': 'sadness',
    '4': 'anger',
    '5': 'disgust',
    '6': 'fear',
    '7': 'guilt',
    '8': 'neutral',
    '9': 'average'
}

print("✓ Библиотеки загружены")

def load_data(file_path):
    path = Path(file_path)
    if not path.exists():
        raise FileNotFoundError(f"Файл не найден: {file_path}")

    suffix = path.suffix.lower()
    print(f"Загрузка {suffix} файла...")

    if suffix == '.csv':
        return pd.read_csv(file_path, encoding='utf-8')
    elif suffix in ['.xls', '.xlsx']:
        return pd.read_excel(file_path, engine='openpyxl')
    elif suffix == '.json':
        return pd.read_json(file_path)
    elif suffix == '.parquet':
        return pd.read_parquet(file_path)
    else:
        raise ValueError(f"Неподдерживаемый формат. Используйте CSV/XLS/JSON/Parquet")

def save_data(df, output_path):
    suffix = Path(output_path).suffix.lower()

    if suffix == '.csv':
        df.to_csv(output_path, index=False, encoding='utf-8')
    elif suffix in ['.xls', '.xlsx']:
        df.to_excel(output_path.replace('.xls', '.xlsx'), index=False, engine='openpyxl')
    elif suffix == '.json':
        df.to_json(output_path, orient='records', force_ascii=False, indent=2)
    elif suffix == '.parquet':
        df.to_parquet(output_path, index=False, compression='snappy')

    print(f"✓ Файл сохранён: {output_path}")

✓ Зависимости установлены
⚠ GPU не обнаружен, будет использоваться CPU
✓ Библиотеки загружены


In [ ]:
def predict_emotions_batch(texts, classifier, batch_size=32):
    all_emotions = []

    for i in tqdm(range(0, len(texts), batch_size), desc="Обработка батчей"):
        batch = texts[i:i + batch_size]
        results = classifier(batch, truncation=True, max_length=512)

        batch_emotions = []
        for r in results:
            if isinstance(r, list):
                label = r[0]['label']
            else:
                label = r['label']

            if isinstance(label, int):
                emotion = EMOTION_MAP.get(label, str(label))
            elif isinstance(label, str):
                emotion = EMOTION_STR_MAP.get(label)

                if emotion is None and label.startswith('LABEL_'):
                    try:
                        label_num = int(label.split('_')[1])
                        emotion = EMOTION_MAP.get(label_num, label)
                    except:
                        emotion = label

                if emotion is None:
                    emotion = label
            else:
                emotion = str(label)

            batch_emotions.append(emotion)

        all_emotions.extend(batch_emotions)

    return all_emotions

def process_dataset(input_path, output_path, batch_size=64):
    df = load_data(input_path)
    print(f"✓ Загружено {len(df)} строк")

    text_col = None
    for col in ['Review', 'text', 'Text', 'review', 'comment']:
        if col in df.columns:
            text_col = col
            break

    if text_col is None:
        raise ValueError("Не найдена колонка с текстом. Ожидаются: 'Review', 'text', 'Text', 'review', 'comment'")

    df[text_col] = df[text_col].astype(str)
    valid_mask = df[text_col].notna() & (df[text_col].str.strip() != '')
    valid_count = valid_mask.sum()

    print(f"✓ Валидных текстов: {valid_count}")

    df['emotion'] = ''

    if valid_count > 0:
        print("\n" + "="*60)
        print("ЗАГРУЗКА МОДЕЛИ")
        print("="*60)
        print("⏳ Загрузка forxtears/airline-emotion-classifier")

        try:
            print("Пробуем загрузить на CPU...")
            classifier = pipeline(
                'text-classification',
                model='forxtears/airline-emotion-classifier',
                device=-1
            )
            print("✓ Модель загружена на CPU")

            '''classifier = pipeline(
                'text-classification',
                model='forxtears/airline-emotion-classifier',
                device=0
            )
            print("✓ Модель загружена на GPU успешно")'''
        except Exception as e:
            print(f"⚠ Ошибка загрузки на GPU: {e}")


        print("\n" + "="*60)
        print(f"ПРЕДСКАЗАНИЕ ЭМОЦИЙ (batch size: {batch_size})")
        print("="*60)

        valid_texts = df.loc[valid_mask, text_col].tolist()
        emotions = predict_emotions_batch(valid_texts, classifier, batch_size)
        df.loc[valid_mask, 'emotion'] = emotions
        print("✓ Предсказания завершены")

    print(f"\n✓ Сохранение в {output_path}...")
    save_data(df, output_path)

    return df

print("✓ Функции определены")

from google.colab import files
import os

print("\n" + "="*60)
print("ЗАГРУЗКА ФАЙЛА")
print("="*60)
print("Выберите ваш dataset файл (CSV/XLSX/JSON/PARQUET):")

uploaded = files.upload()

if not uploaded:
    raise FileNotFoundError("Файл не был загружен")

input_filename = list(uploaded.keys())[0]
output_filename = 'emotion_enriched_' + input_filename

print(f"✓ Загружен файл: {input_filename}")

✓ Функции определены

ЗАГРУЗКА ФАЙЛА
Выберите ваш dataset файл (CSV/XLSX/JSON/PARQUET):


Saving s7_airlines_reviews_with_sentiment.csv to s7_airlines_reviews_with_sentiment (2).csv
✓ Загружен файл: s7_airlines_reviews_with_sentiment (2).csv


In [ ]:
try:
    df = process_dataset(
        input_path=input_filename,
        output_path=output_filename,
        batch_size=32
    )

    print("\n" + "="*60)
    print("РЕЗУЛЬТАТЫ")
    print("="*60)
    print(f"✓ Всего строк обработано: {len(df)}")
    print(f"✓ Валидных предсказаний: {(df['emotion'] != '').sum()}")

    if (df['emotion'] != '').any():
        print("\n📊 Распределение эмоций:")
        emotion_counts = df[df['emotion'] != '']['emotion'].value_counts()
        for emotion, count in emotion_counts.items():
            percentage = (count / len(df[df['emotion'] != ''])) * 100
            bar = '█' * int(percentage / 2)
            print(f"  {emotion:12s}: {count:4d} ({percentage:5.1f}%) {bar}")

    print("\n📝 Примеры предсказаний:")
    text_col = [c for c in ['Review', 'text', 'Text', 'review', 'comment'] if c in df.columns][0]
    sample = df[df['emotion'] != ''].head(10)
    for idx, row in sample.iterrows():
        text_preview = row[text_col][:70] + "..." if len(row[text_col]) > 70 else row[text_col]
        print(f"  [{row['emotion']:10s}] {text_preview}")

    print("\n" + "="*60)
    print("✓ УСПЕШНО ЗАВЕРШЕНО! 🎉")
    print(f"Результат сохранён: {output_filename}")
    print("="*60)

    files.download(output_filename)

except Exception as e:
    print(f"\n✗ ОШИБКА: {e}")
    import traceback
    traceback.print_exc()

Загрузка .csv файла...
✓ Загружено 8740 строк
✓ Валидных текстов: 8740

ЗАГРУЗКА МОДЕЛИ
⏳ Загрузка forxtears/airline-emotion-classifier
Пробуем загрузить на CPU...


Device set to use cpu


✓ Модель загружена на CPU

ПРЕДСКАЗАНИЕ ЭМОЦИЙ (batch size: 32)


Обработка батчей: 100%|██████████| 274/274 [1:14:01<00:00, 16.21s/it]


✓ Предсказания завершены

✓ Сохранение в emotion_enriched_s7_airlines_reviews_with_sentiment (2).csv...
✓ Файл сохранён: emotion_enriched_s7_airlines_reviews_with_sentiment (2).csv

РЕЗУЛЬТАТЫ
✓ Всего строк обработано: 8740
✓ Валидных предсказаний: 8740

📊 Распределение эмоций:
  joy         : 3565 ( 40.8%) ████████████████████
  disgust     : 2070 ( 23.7%) ███████████
  anger       : 1610 ( 18.4%) █████████
  fear        :  460 (  5.3%) ██
  interest    :  345 (  3.9%) █
  guilt       :  230 (  2.6%) █
  neutral     :  230 (  2.6%) █
  surprise    :  230 (  2.6%) █

📝 Примеры предсказаний:
  [joy       ] Выгодно слетали на свадьбу к двоюродному брату в Сочи. У нас получилас...
  [anger     ] Купила билет на рейс авиакомпании S7 по тарифу эконом базовый. Была ув...
  [disgust   ] Нормально с котом долетели из Иркутска на выставку в Москву. У авиаком...
  [joy       ] Нравится, что с С7 получается выгодно получается летать. Во-первых про...
  [joy       ] Часто за последний год летал С7

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>